<a href="https://colab.research.google.com/github/park-geun-hyeong/Computer_Vision/blob/main/paper/Resnet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision

import os
import numpy as np
import cv2 
import matplotlib.pyplot as plt

# Modeling

## Conv Start

In [21]:
def conv_start():
    return nn.Sequential(
        nn.Conv2d(3,64, kernel_size=7, stride=2, padding=4),
        nn.BatchNorm2d(64),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size = 3, stride = 2)
    )

## Bottle Neck

In [10]:
def bottleneck_block(in_dim, mid_dim, out_dim, down=False):
    layers=[]
    if down:
        layers.append(nn.Conv2d(in_dim, mid_dim, kernel_size=1, stride=2, padding=0))  
    else:
        layers.append(nn.Conv2d(in_dim, mid_dim, kernel_size=1, stride=1, padding=0))

    layers.extend([
        nn.BatchNorm2d(mid_dim),
        nn.ReLU(inplace=True),
        nn.Conv2d(mid_dim, mid_dim, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(mid_dim),
        nn.ReLU(inplace=True),
        nn.Conv2d(mid_dim, out_dim, kernel_size=1, stride=1, padding=0),
        nn.BatchNorm2d(out_dim)
    ])

    return nn.Sequential(*layers)




In [12]:
bottleneck_block(64,64,256)

Sequential(
  (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU(inplace=True)
  (6): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
  (7): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [26]:
class Bottleneck(nn.Module):

    def __init__(self, in_dim, mid_dim, out_dim, down:bool=False, starting:bool=False) -> None:
        super(Bottleneck, self).__init__()

        if starting:
            down = False

        self.block = bottleneck_block(in_dim, mid_dim, out_dim)
        self.relu = nn.ReLU(inplace=True)
        if down:
            conn_layer = nn.Conv2d(in_dim, out_dim, kernel_size = 1, stride=2, padding=0)
        else:
            conn_layer = nn.Conv2d(in_dim, out_dim, kernel_size=1, stride=1, padding=0)

        self.changedim = nn.Sequential(conn_layer, nn.BatchNorm2d(out_dim))


    def forward(self, x):
        identity = self.changedim(x)
        x = self.block(x)
        x = x + identity
        x = self.relu(x)

        return x      

In [33]:
def make_layer(in_dim, mid_dim, out_dim, repeats , starting:bool=False):
    layers=[]
    layers.append(Bottleneck(in_dim, mid_dim, out_dim, down=True ))
    for _ in range(1, repeats):
        layers.append(Bottleneck(out_dim, mid_dim, out_dim, down=False))

    return nn.Sequential(*layers)

## ResNet

In [35]:
class ResNet(nn.Module):

    def __init__(self, repeats:list = [3,6,4,3], num_classes=1000):
        super(ResNet, self).__init__()

        self.num_classes = num_classes
        self.conv1 = conv_start()

        base_dim = 64
        self.conv2 = make_layer(base_dim, base_dim, base_dim*4,repeats=repeats[0], starting=True)
        self.conv3 = make_layer(base_dim*4, base_dim*2, base_dim*8,repeats=repeats[1])
        self.conv4 = make_layer(base_dim*8, base_dim*4, base_dim*16,repeats=repeats[2])
        self.conv5 = make_layer(base_dim*16, base_dim*6, base_dim*32,repeats=repeats[3])

        self.avgpool = nn.AvgPool2d(kernel_size=7, stride=1)
        self.fc_layer = nn.Linear(base_dim*32, self.num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.avgpool(x)

        x = x.view(x.size(0), -1)
        x = self.fc_layer(x)

        return x



In [36]:
ResNet()

ResNet(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(4, 4))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Bottleneck(
      (block): Sequential(
        (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU(inplace=True)
        (6): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
        (7): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (relu): ReLU(inplace=True)
      (changedim): Sequentia